## Rte1:

### Model : 9a275f9320714938a25097c942b35477

In [4]:
#from onlymodel import show_metric
#from metricsyso import *


# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from metricsyso import *
from tqdm import tqdm

def show_metric(truthpath, predpath, savepath):
    # Créez le répertoire de sauvegarde s'il n'existe pas
    os.makedirs(savepath, exist_ok=True)
    pred_filename = os.path.splitext(os.path.basename(predpath))[0]
    
    conv_dict1 = {"1": 0,"2": 1,"3": 0,"4": 2,"5": 2, "6": 3,"7": 8,"12": 0,"13": 0,"15": 0,"50": 4,"51": 4,
            "52": 4, "53": 4,"54": 4, "55": 4, "64": 4,"72": 7, "101": 5,"102": 5,"103": 5, "104": 5,
        "105": 5,"106": 5,"107": 5, "108": 5,"109": 5,"110": 5,"111": 5,"120": 5,"121": 5,"150": 6, "151": 6,
    "152": 6,"53": 6, "154": 6,"155": 6, "160": 6, "161": 6, "185": 6, "196": 5,
    "197": 5,"200": 6,"201": 6, "202": 6,"211": 6,"212": 6,"213": 6, "221": 6, "222": 6,"223": 6
    }
    model1 = {"0":1, "1":2, "2":5, "3":6, "4":59, "5":196, "6": 211, "7":72}

#

    direct_conv1={k:j for k, v in conv_dict1.items() for  i, j in model1.items()  if int(i) == v }


    # Exemple d'utilisation
    rte1_pred=read_file_class(predpath)
    rte1_truth=read_file_class(truthpath)
    pred_class = num_points(rte1_pred)
    gtruth_class = num_points(rte1_truth)


    gtruth_cleanead = [direct_conv1.get(str(k)) if direct_conv1.get(str(k)) is not None else k for k in gtruth_class]

    rte1_ground_truth =np.array(gtruth_cleanead)
    rte1_predictions = np.array(pred_class)
    
    print("len gtruth_class",len(gtruth_class))
    print("gtruth unique classes",np.unique(gtruth_class))    
    print("") 
    print("len rte1_gtruth_class", len(rte1_ground_truth))
    print("unique gtruth classes merged classe", np.unique(rte1_ground_truth))
    
    print("")
    print(f"len pred_class{len(rte1_predictions)}")
    print(f" unique classe predictions{np.unique(rte1_predictions)}")
    print("")

    unique_classes =  np.unique(np.concatenate((rte1_ground_truth, rte1_predictions)))
    clas_name = [str(name) for name in tqdm(unique_classes)]

    default_dict = {"1":"default","2":"ground","5": "vegetation", 
                "6":"building","7": "wall", 
                "196": "structure","211":"cable"
                }

    conf_matrix = compute_confusion_matrix(rte1_ground_truth, rte1_predictions)
    
    print(conf_matrix.shape[0])
    print(len(clas_name))
    if conf_matrix.shape[0] != len(clas_name) or conf_matrix.shape[1] != len(clas_name):
        raise ValueError("Mismatch between confusion matrix dimensions and class names")
          
    #conf_matrix = confusion_matrix(rte1_ground_truth, rte1_predictions)

    # Sauvegarder la matrice de confusion dans un fichier Excel
    with pd.ExcelWriter(os.path.join(savepath, f"conf_matrix_{pred_filename}.xlsx")) as writer:

        #df = pd.DataFrame(conf_matrix, index=clas_name, columns=clas_name)
        conf_matrix.rename(columns = default_dict, index = default_dict, inplace = True)
        conf_matrix.to_excel(writer, index=True)
        print(f"Fichier sauvegardé avec succès: {os.path.join(savepath, f'conf_matrix_{pred_filename}.xlsx')}")

    # Afficher et sauvegarder la matrice de confusion
        
    #ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=clas_name)
    """dis=ConfusionMatrixDisplay.from_predictions(rte1_ground_truth, rte1_predictions, labels = clas_name )
    mask = np.eye(len(conf_matrix))

    fig, ax = plt.subplots(figsize=(30, 10))
    #ConfusionMatrixDisplay(conf_matrix, display_labels=clas_name).plot(ax=ax)
    dis.plot(ax=ax)
    ax.imshow(mask, cmap='Blues', interpolation='nearest', alpha=0.5)

    plt.savefig(os.path.join(savepath, f"conf_matrix_{pred_filename}.png"))
    plt.show()"""
    
    
    #========================== metrics=============================================#
    
    metrics=Metrics(rte1_ground_truth,rte1_predictions)
    ## calling a methods for computing metrics
    m=metrics.compute_metrics()

    ## for displaying as frame

    fr = metrics.metrics_frame(m)
    wg = metrics.weighted_metrics()


    # Sauvegarder la matrice de confusion dans un fichier Excel
    with pd.ExcelWriter(os.path.join(savepath, f"{pred_filename}_metric_frame.xlsx")) as writer:
        
        fr.rename(columns =default_dict,index = default_dict, inplace = True)
        fr.to_excel(writer, index=True)
        print(f"Fichier sauvegardé avec succès: {os.path.join(savepath, f'{pred_filename}_metric_frame.xlsx')}")


    # Sauvegarder la matrice de confusion dans un fichier Excel
    with pd.ExcelWriter(os.path.join(savepath, f"{pred_filename}_weighted_m.xlsx")) as writer:
        wg.rename(columns =default_dict, index = default_dict, inplace = True)
        wg.to_excel(writer, index=True)
        print(f"Fichier sauvegardé avec succès: {os.path.join(savepath, f'{pred_filename}_weighted_m.xlsx')}")
        
truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/db3ea63f541a4d9fbe68bd96f98a86ae/preds/000006.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/metrics"
show_metric(truthpath,predpath,savepath)

100%|██████████| 113550333/113550333 [00:12<00:00, 8940892.01it/s] 


len gtruth_class 113550333
gtruth unique classes [  2   3   4   5   6  14  50  51  56  70  71  72  79 106 107 111 150 151
 161 196 197 211 212 213]

len rte1_gtruth_class 113550333
unique gtruth classes merged classe [  1   2   5   6  14  56  59  70  71  72  79 196 211]

len pred_class113550333
 unique classe predictions[  1   2   5   6  59  72 196]



100%|██████████| 13/13 [00:00<?, ?it/s]


len ground_truth 113550333
len pred 113550333


100%|██████████| 113550333/113550333 [06:42<00:00, 282155.16it/s]


13
13
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/metrics\conf_matrix_000006.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/metrics\000006_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/metrics\000006_weighted_m.xlsx


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import font, filedialog, messagebox
import json
from metricsyso import *
import pandas as pd
import time
from tqdm import tqdm

class MyApp():
    def __init__(self,windows):
        self.windows = windows
        windows.title("My App")
        self.conf_df = None
        self.df_m =None 
        self.df_w =None
        self.default_dict = default_dict = {"1":"default","2":"ground","5": "vegetation", 
                "6":"building","7": "wall", 
                "196": "structure","211":"cable"
                }
        #self.button = Button(windows,text="Display confsusion matrix", command =self.compute_metrics).place(x=60, y=45,height = 50, width = 180)
        
        # Create buttons to compute metrics
        self.compute_button = tk.Button(windows, text="Compute Metrics",command=lambda: self.compute_metrics)
        self.compute_button.place(x=60, y=45,height = 50, width = 180)
       
        
        self.button1 = Button(windows,text="Display confsusion matrix", command =self.display_confmatrix).place(x=60, y=200,height = 50, width = 180)
      
        self.button2 = Button(windows,text="Display metrics", command =self.display_df_metrics).place(x=60, y=260,height = 50, width = 180)
       
        self.buton3 = Button(windows,text="Display weighted metrics", command =self.display_weightetd_metrics).place(x=60, y=340,height = 50, width =180)
       
        self.gtruth_file_entry = Entry(windows, width=50)
        self.gtruth_file_entry.pack()
        self.gtruth_file_btn = tk.Button(windows, text="Load Ground Truth File", command=lambda: self.gtruth_file_entry.insert(tk.END, self.browse_file("Select Ground Truth File")))
        self.gtruth_file_btn.pack()
        
        self.pred_file_entry = Entry(windows, width=50)
        self.pred_file_entry.pack()
        self.pred_file_btn = tk.Button(windows, text="Load Prediction File", command=lambda: self.pred_file_entry.insert(tk.END, self.browse_file("Select Prediction File")))
        self.pred_file_btn.pack()
        
        self.json_file_entry = Entry(windows, width=50)
        self.json_file_entry.pack()
        self.json_file_btn = tk.Button(windows, text="Load JSON File", command=lambda: self.json_file_entry.insert(tk.END, self.browse_file("Select JSON File")))
        self.json_file_btn.pack()
        
        # place the pbbar
        self.pb = ttk.Progressbar(windows, orient='horizontal',    mode='indeterminate',    length=280)
        self.pb.place(x=60, y=100,height = 20, width = 180)
        
        self.status_label = tk.Label(windows, text="", padx=20, pady=10)
        self.status_label.pack()
        
        self.save_button = tk.Button(windows, text="Save to File", command=self.save_to_file)
        self.save_button.pack(pady=10)
        
        self.text_widget = tk.Text(windows, wrap=tk.WORD)
        self.text_widget.pack(padx=20, pady=20, fill="both", expand=True)

    def browse_file(self,title):
        filepath = filedialog.askopenfilename(title=title, filetypes=(("Laz or Las files", "*.laz *.las"), ("All files", "*.*")))
        return filepath


    def tree_display(self,df):
            df_list=list(df.columns.values)
            df_rset=df.to_numpy().tolist()
            df_tree=ttk.Treeview(self.windows,columns=df_list)
            df_tree.pack()                                      
            for i in df_list:
                df_tree.column(i,width=100,anchor='c')
                df_tree.heading(i,text=i)
            for dt in df_rset:
                v=[r for r in dt]
                df_tree.insert('','end', values=v)
            df_tree.tag_configure('ttk', background='yellow')

    def compute_metrics(self):
        
        gtruth_file =self.gtruth_file_entry.get()
        pred_file = self.pred_file_entry.get()
        json_file = self.json_file_entry.get()
        
        
        # Reset pb bar
        self.pb['value'] = 0
        self.windows.update_idletasks()
        
        # step 1: Load Json file
        op = JsonFile(json_file)
        conv_dict = op.convdict()
        conv_class = op.convclass()
        truth_class = op.conv_to_Truclass(conv_dict, conv_class)
        self.pb['value'] = 20
        self.windows.update_idletasks() 

        # Step 1 : read gtruth file and  pred file
        gtruth = read_file_class(gtruth_file)
        prediction = read_file_class(pred_file)
        count_pred = num_points(prediction)
        count_gtruth = num_points(gtruth)
        self.pb['value'] = 40
        self.windows.update_idletasks()
        
        # step 3 : convert gtruth classes
        convertor = truth_class
        gtruth_conv = [convertor.get(str(k)) if convertor.get(str(k)) is not None else k for k in count_gtruth]
        self.pb['value'] = 60
        self.windows.update_idletasks()

        # step 4 : conpute confusion matrix
        input_pred = np.array(count_pred)
        input_gtruth = np.array(gtruth_conv)
        unique_classes =  np.unique(np.concatenate((input_gtruth, input_pred )))
        self.clas_name = [str(name) for name in tqdm(unique_classes)]
        self.conf_df= compute_confusion_matrix(input_gtruth, input_pred)
        self.conf_df.rename(columns = self.default_dict, index = self.default_dict, inplace = True)
        self.pb['value'] = 80
        self.windows.update_idletasks()
        
        # Step 5 compute metrics
        m = Metrics(input_gtruth, input_pred)
        metrics_fr = m.compute_metrics()
        self.df_m =m.metrics_frame(metrics_fr)
        self.df_w = m.weighted_metrics()
        self.pb['value'] = 100
        self.windows.update_idletasks()

    

    # displayinf data frame witrh a treeView
    
    def display_confmatrix(self):
        self.tree_display(self.conf_df)

    def display_df_metrics(self):
        self.tree_display(self.df_m)

    def display_weightetd_metrics(self):
        self.tree_display(self.df_w)
        
        
    """def self.save_to_file (self):
        path = asksaveasfile()
        with pd.ExcelWriter(path, engine ="openpyxl", mode ="a") as writer:
            self.conf_df.to_excel(writer, index=True)
            print(f"Fichier sauvegardé avec succès: {os.path.join(savepath, f'conf_matrix_{pred_filename}.xlsx')}")"""
            

    def save_to_file(self):
        
        file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if file_path:
            
            try:                
                with open (file_path,'w') as file:
                    
                    text_content = self.text_widget.get("1.0", "end-1c")
                file.write(text_content)
                     
            except Exception as e:
                self.status_label.config(text=f"Error saving file: {str(e)}")
        
        
        
        
        
windows =Tk()
my_gui = MyApp(windows)

windows.geometry('900x600')
windows.mainloop()

## Rte2:

### Model : d167dd0644634a8eaee6241ed7c375ce

In [5]:
from SaveMetrics import show_metrics
from metricsyso import *


In [6]:

truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/d167dd0644634a8eaee6241ed7c375ce/preds/000006.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics"
show_metric(truthpath,predpath,savepath)

100%|██████████| 113550333/113550333 [00:11<00:00, 10048648.65it/s]


len gtruth_class 113550333
gtruth unique classes [  2   3   4   5   6  14  50  51  56  70  71  72  79 106 107 111 150 151
 161 196 197 211 212 213]

len rte1_gtruth_class 113550333
unique gtruth classes merged classe [  1   2   5   6  14  56  59  70  71  72  79 196 211]

len pred_class113550333
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 13/13 [00:00<?, ?it/s]


len ground_truth 113550333
len pred 113550333


100%|██████████| 113550333/113550333 [06:39<00:00, 283953.28it/s]


13
13
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics\conf_matrix_000006.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics\000006_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics\000006_weighted_m.xlsx


### Model : b478f486891d47aca0ad2b5f116d48c2

In [7]:

truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008_15/b478f486891d47aca0ad2b5f116d48c2/preds/000006.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008_15/metrics"
show_metric(truthpath,predpath,savepath)

100%|██████████| 113550333/113550333 [00:11<00:00, 9946526.73it/s] 


len gtruth_class 113550333
gtruth unique classes [  2   3   4   5   6  14  50  51  56  70  71  72  79 106 107 111 150 151
 161 196 197 211 212 213]

len rte1_gtruth_class 113550333
unique gtruth classes merged classe [  1   2   5   6  14  56  59  70  71  72  79 196 211]

len pred_class113550333
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 13/13 [00:00<?, ?it/s]


len ground_truth 113550333
len pred 113550333


100%|██████████| 113550333/113550333 [06:43<00:00, 281230.67it/s]


13
13
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008_15/metrics\conf_matrix_000006.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008_15/metrics\000006_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008_15/metrics\000006_weighted_m.xlsx


## Model : 3285002e42f64f02af799c9ffdc4ffb6

In [8]:

truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-16/3285002e42f64f02af799c9ffdc4ffb6/preds/000006.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-16/metrics"
show_metric(truthpath,predpath,savepath)

100%|██████████| 113550333/113550333 [00:11<00:00, 9983511.33it/s] 


len gtruth_class 113550333
gtruth unique classes [  2   3   4   5   6  14  50  51  56  70  71  72  79 106 107 111 150 151
 161 196 197 211 212 213]

len rte1_gtruth_class 113550333
unique gtruth classes merged classe [  1   2   5   6  14  56  59  70  71  72  79 196 211]

len pred_class113550333
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 13/13 [00:00<?, ?it/s]


len ground_truth 113550333
len pred 113550333


100%|██████████| 113550333/113550333 [06:42<00:00, 281885.12it/s]


13
13
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-16/metrics\conf_matrix_000006.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-16/metrics\000006_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-16/metrics\000006_weighted_m.xlsx


## Model : 99385bdd1b4245d6813754f478bd5f2c

In [3]:
from SaveMetrics import show_metrics
from metricsyso import *

truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/99385bdd1b4245d6813754f478bd5f2c/preds/000006.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/metrics"
show_metrics(truthpath,predpath,savepath)

100%|██████████| 113550333/113550333 [00:11<00:00, 10016476.06it/s]


len gtruth_class 113550333
gtruth unique classes [  2   3   4   5   6  14  50  51  56  70  71  72  79 106 107 111 150 151
 161 196 197 211 212 213]

len rte2_gtruth_class 113550333
unique gtruth classes merged classe [  1   2   5   6  14  56  70  71  79 196 211]

len pred_class113550333
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 11/11 [00:00<?, ?it/s]


len ground_truth 113550333
len pred 113550333


100%|██████████| 113550333/113550333 [06:37<00:00, 285477.53it/s]


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/metrics\conf_matrix_000006.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/metrics\000006_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/metrics\000006_weighted_m.xlsx


## Dalle 00004

## Model : 3285002e42f64f02af799c9ffdc4ffb6

In [1]:
from SaveMetrics import show_metrics
from metricsyso import *

In [3]:
truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000004.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/3285002e42f64f02af799c9ffdc4ffb6/preds/000004.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics04"
show_metrics(truthpath,predpath,savepath)

100%|██████████| 58898000/58898000 [00:06<00:00, 9372412.11it/s] 


len gtruth_class 58898000
gtruth unique classes [  2   3   5   6  14  56  60  72 111 161 196 197 200 211 212 213]

len rte2_gtruth_class 58898000
unique gtruth classes merged classe [  1   2   5   6  14  56  60 196 211]

len pred_class58898000
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 9/9 [00:00<?, ?it/s]


len ground_truth 58898000
len pred 58898000


100%|██████████| 58898000/58898000 [03:27<00:00, 284206.59it/s]


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics04\conf_matrix_000004.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics04\000004_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics04\000004_weighted_m.xlsx


### Model : d167dd0644634a8eaee6241ed7c375ce

In [4]:

truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000004.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/d167dd0644634a8eaee6241ed7c375ce/preds/000004.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics04"
show_metrics(truthpath,predpath,savepath)

100%|██████████| 58898000/58898000 [00:06<00:00, 9815205.39it/s] 


len gtruth_class 58898000
gtruth unique classes [  2   3   5   6  14  56  60  72 111 161 196 197 200 211 212 213]

len rte2_gtruth_class 58898000
unique gtruth classes merged classe [  1   2   5   6  14  56  60 196 211]

len pred_class58898000
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 9/9 [00:00<?, ?it/s]


len ground_truth 58898000
len pred 58898000


100%|██████████| 58898000/58898000 [03:30<00:00, 280310.51it/s]


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics04\conf_matrix_000004.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics04\000004_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics04\000004_weighted_m.xlsx


## Dalle 000001

## Model : 3285002e42f64f02af799c9ffdc4ffb6

In [5]:
truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000001.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/3285002e42f64f02af799c9ffdc4ffb6/preds/000001.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics01"
show_metrics(truthpath,predpath,savepath)

100%|██████████| 94723017/94723017 [00:09<00:00, 9518763.70it/s] 


len gtruth_class 94723017
gtruth unique classes [  2   3   4   5   6  14  56  60  72  79 103 104 105 107 109 151 153 154
 155 196 197 200 211 212 213]

len rte2_gtruth_class 94723017
unique gtruth classes merged classe [  1   2   5   6  14  56  60  79 196 211]

len pred_class94723017
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 10/10 [00:00<?, ?it/s]


len ground_truth 94723017
len pred 94723017


100%|██████████| 94723017/94723017 [05:37<00:00, 280591.17it/s]


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics01\conf_matrix_000001.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics01\000001_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_008-15v2/metrics01\000001_weighted_m.xlsx


### Model : d167dd0644634a8eaee6241ed7c375ce

In [6]:
truthpath = r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000001.laz"
## Use the predited files path 
predpath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/d167dd0644634a8eaee6241ed7c375ce/preds/000001.laz"
savepath = r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics01"
show_metrics(truthpath,predpath,savepath)

100%|██████████| 94723017/94723017 [00:09<00:00, 9882645.88it/s] 


len gtruth_class 94723017
gtruth unique classes [  2   3   4   5   6  14  56  60  72  79 103 104 105 107 109 151 153 154
 155 196 197 200 211 212 213]

len rte2_gtruth_class 94723017
unique gtruth classes merged classe [  1   2   5   6  14  56  60  79 196 211]

len pred_class94723017
 unique classe predictions[  1   2   5   6 196 211]



100%|██████████| 10/10 [00:00<00:00, 10003.11it/s]


len ground_truth 94723017
len pred 94723017


100%|██████████| 94723017/94723017 [05:41<00:00, 277731.00it/s]


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics01\conf_matrix_000001.xlsx


c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics01\000001_metric_frame.xlsx
Fichier sauvegardé avec succès: E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3_sans_intensity/metrics01\000001_weighted_m.xlsx
